In [3]:
%load_ext Cython

In [12]:
# !pip install line_profiler==3.3.1
%load_ext line_profiler

In [2]:
from tqdm import tqdm
import numpy as np
import random
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import string
from torch.utils.data import Dataset, DataLoader
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
nltk.download('stopwords')
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\czy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\czy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
! unzip /root/nltk_data/corpora/wordnet.zip -d /root/nltk_data/corpora/

Archive:  /root/nltk_data/corpora/wordnet.zip
   creating: /root/nltk_data/corpora/wordnet/
  inflating: /root/nltk_data/corpora/wordnet/lexnames  
  inflating: /root/nltk_data/corpora/wordnet/data.verb  
  inflating: /root/nltk_data/corpora/wordnet/index.adv  
  inflating: /root/nltk_data/corpora/wordnet/adv.exc  
  inflating: /root/nltk_data/corpora/wordnet/index.verb  
  inflating: /root/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /root/nltk_data/corpora/wordnet/data.adj  
  inflating: /root/nltk_data/corpora/wordnet/index.adj  
  inflating: /root/nltk_data/corpora/wordnet/LICENSE  
  inflating: /root/nltk_data/corpora/wordnet/citation.bib  
  inflating: /root/nltk_data/corpora/wordnet/noun.exc  
  inflating: /root/nltk_data/corpora/wordnet/verb.exc  
  inflating: /root/nltk_data/corpora/wordnet/README  
  inflating: /root/nltk_data/corpora/wordnet/index.sense  
  inflating: /root/nltk_data/corpora/wordnet/data.noun  
  inflating: /root/nltk_data/corpora/wordnet/data.adv  


In [3]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

In [4]:
seed_everything(42)

42

In [5]:
MAX_LENGTH = 30
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [6]:
train= pd.read_csv('./train.csv')
test= pd.read_csv("./test.csv")

In [7]:
train.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

# Preprocessing without optimization

In [8]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', string)

def clean_sentence(sentence):
    # remove URLS 
    sentence = re.sub(r'http\S+', ' ', sentence)
    # remove emoji's
    sentence = remove_emoji(sentence)
    # remove punctuation
    sentence = re.sub("[^0-9A-Za-z ]", "" , sentence)
    # remove double spaces
    sentence = sentence.replace('  ',"")
    
    return sentence.strip()

def remove_stopwords(tokens, stopwords):
    clean_token = [word for word in tokens if word not in stopwords]
    return clean_token

# lemmalization 
def lemmatize(tokens, lemma):
    lemmatized_tokens = [lemma.lemmatize(token, pos = 'v') for token in tokens]
    return lemmatized_tokens



In [9]:
# fix sentence length
def trunc_padding(sentence):
    modify_sentence = sentence.copy()
    if len(modify_sentence) >= MAX_LENGTH:
        modify_sentence = modify_sentence[:MAX_LENGTH]
    else:
        modify_sentence.extend(list(["0"] * (MAX_LENGTH - len(modify_sentence))))
    return modify_sentence

In [10]:
stopwords = nltk.corpus.stopwords.words('english')
lemma = WordNetLemmatizer()

def processing(df, stopwords, lemma):
    df['text'] = df['text'].apply(lambda sentence: sentence.lower())
    df['text'] = df['text'].apply(lambda sentence: clean_sentence(sentence))
    # tokenization
    df['text'] = df['text'].apply(lambda sentence: sentence.split())
    # remove stopwords
    df['text'] = df['text'].apply(lambda sentence: remove_stopwords(sentence, stopwords))
    # lemmalization 
    df['text'] = df['text'].apply(lambda sentence: lemmatize(sentence, lemma))
    # sentence length before padding
    df['length'] = df['text'].apply(lambda x: len(x))
    # fix sentence length
    df['text'] = df['text'].apply(lambda sentence: trunc_padding(sentence))
    # sentence length after padding
    df['length_padding'] = df['text'].apply(lambda x: len(x))

In [13]:
%lprun -f processing processing(train, stopwords, lemma)

Timer unit: 1e-07 s

Total time: 3.35363 s

Could not find file C:\Users\czy\AppData\Local\Temp\ipykernel_17644\1383951964.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           
     5         1      52942.0  52942.0      0.2  
     6         1     902872.0 902872.0      2.7  
     7                                           
     8         1      80908.0  80908.0      0.2  
     9                                           
    10         1    1381567.0 1381567.0      4.1  
    11                                           
    12         1   30905428.0 30905428.0     92.2  
    13                                           
    14         1      32408.0  32408.0      0.1  
    15                                           
    16         1     148931.0 148931.0      0.4  
    17         

In [14]:
%lprun -f processing processing(test, stopwords, lemma)

Timer unit: 1e-07 s

Total time: 0.377591 s

Could not find file C:\Users\czy\AppData\Local\Temp\ipykernel_17644\1383951964.py
Are you sure you are running this program from the same directory
that you ran the profiler from?
Continuing without the function's contents.

Line #      Hits         Time  Per Hit   % Time  Line Contents
     4                                           
     5         1      21408.0  21408.0      0.6  
     6         1     323353.0 323353.0      8.6  
     7                                           
     8         1      39792.0  39792.0      1.1  
     9                                           
    10         1     611524.0 611524.0     16.2  
    11                                           
    12         1    2684292.0 2684292.0     71.1  
    13                                           
    14         1      17390.0  17390.0      0.5  
    15                                           
    16         1      61179.0  61179.0      1.6  
    17          

# Preprocessing with Cython optimization

In [4]:
%%cython 
# cython: linetrace=True
# cython: binding=True
# distutils: define_macros=CYTHON_TRACE_NOGIL=1
# cython: profile=True
from tqdm import tqdm
import numpy as np
import random
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import string
from torch.utils.data import Dataset, DataLoader
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import itertools
nltk.download('stopwords')
nltk.download("wordnet")
cdef seed_everything(int seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

seed_everything(42)
cdef int MAX_LENGTH = 30
cdef object device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

cdef clean_sentence(object sentence):
    # remove URLS 
    sentence = re.sub(r'http\S+', ' ', sentence)
    # remove emoji's
    cdef object emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    sentence = emoji_pattern.sub(r' ', sentence)

    # remove punctuation
    sentence = re.sub("[^0-9A-Za-z ]", "" , sentence)
    # remove double spaces
    sentence = sentence.replace('  ',"")
    
    return sentence.strip()

cdef remove_stopwords(list tokens, object stopwords):
    cdef list clean_token = [word for word in tokens if word not in stopwords]
    return clean_token

# lemmalization 
cdef lemmatize(list tokens, object lemma):
    cdef list lemmatized_tokens = [lemma.lemmatize(token, pos = 'v') for token in tokens]
    return lemmatized_tokens

cdef trunc_padding(list sentence):
    sentence = list(itertools.islice(itertools.chain(sentence, itertools.repeat('0')), 0, MAX_LENGTH)) # fix sentence length
    return sentence

cdef object stopwords = nltk.corpus.stopwords.words('english')
cdef object lemma = WordNetLemmatizer()

def processing(df: pd.DataFrame, object stopwords, object lemma):
    df['text'] = df['text'].map(lambda sentence: sentence.lower())
    df['text'] = df['text'].map(lambda sentence: clean_sentence(sentence))
    # tokenization 123123
    df['text'] = df['text'].map(lambda sentence: sentence.split())
    # remove stopwords
    df['text'] = df['text'].map(lambda sentence: remove_stopwords(sentence, stopwords))
    # lemmalization 
    df['text'] = df['text'].map(lambda sentence: lemmatize(sentence, lemma))
    # sentence length before padding
    df['length'] = df['text'].map(lambda x: len(x))
    # fix sentence length
    df['text'] = df['text'].map(lambda sentence: trunc_padding(sentence))
    # sentence length after padding
    df['length_padding'] = df['text'].map(lambda x: len(x))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\czy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\czy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [6]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download("wordnet")
train= pd.read_csv('./train.csv')
test= pd.read_csv("./test.csv")
stopwords = set(stopwords.words('english'))
lemma = WordNetLemmatizer()
%reload_ext line_profiler
%lprun -f processing processing(train, stopwords, lemma)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\czy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\czy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Timer unit: 1e-07 s

Total time: 3.26317 s
File: C:\Users\czy\.ipython\cython\_cython_magic_9b41f3ce78be3cc6a616579f23709c5e.pyx
Function: processing at line 76

Line #      Hits         Time  Per Hit   % Time  Line Contents
    76                                           def processing(df: pd.DataFrame, object stopwords, object lemma):
    77         1      51025.0  51025.0      0.2      df['text'] = df['text'].map(lambda sentence: sentence.lower())
    78         1     866269.0 866269.0      2.7      df['text'] = df['text'].map(lambda sentence: clean_sentence(sentence))
    79                                               # tokenization 123123
    80         1      80244.0  80244.0      0.2      df['text'] = df['text'].map(lambda sentence: sentence.split())
    81                                               # remove stopwords
    82         1      94349.0  94349.0      0.3      df['text'] = df['text'].map(lambda sentence: remove_stopwords(sentence, stopwords))
    83              

In [7]:
%lprun -f processing processing(test, stopwords, lemma)

Timer unit: 1e-07 s

Total time: 0.323276 s
File: C:\Users\czy\.ipython\cython\_cython_magic_9b41f3ce78be3cc6a616579f23709c5e.pyx
Function: processing at line 76

Line #      Hits         Time  Per Hit   % Time  Line Contents
    76                                           def processing(df: pd.DataFrame, object stopwords, object lemma):
    77         1      24311.0  24311.0      0.8      df['text'] = df['text'].map(lambda sentence: sentence.lower())
    78         1     303374.0 303374.0      9.4      df['text'] = df['text'].map(lambda sentence: clean_sentence(sentence))
    79                                               # tokenization 123123
    80         1      57234.0  57234.0      1.8      df['text'] = df['text'].map(lambda sentence: sentence.split())
    81                                               # remove stopwords
    82         1      51662.0  51662.0      1.6      df['text'] = df['text'].map(lambda sentence: remove_stopwords(sentence, stopwords))
    83             

# Comparison
processing training data without optimization takes:  3.35363 s <br>
processing training data with Cython optimization takes:  3.26317 s <br>
The speed up is : 3.35363 / 3.26317 ≈ 1.03 <br>
The relative speed up is 3 %
<br><br>
processing testing data without optimization takes:  0.377591 s<br>
processing testing data with Cython optimization takes:  0.323276 s <br>
The speed up is : 0.377591 / 0.323276 ≈ 1.17<br>
The relative speed up is 17 %

